In [ ]:
# Import the DuckDB library for managing and querying databases     
import duckdb as ddb

In [ ]:
# Install and load the HTTPFS extension in DuckDB
# HTTPFS allows DuckDB to access data over HTTP or from cloud storage (e.g., S3 buckets)
ddb.sql("INSTALL httpfs; LOAD httpfs")

In [ ]:
# Connect to a DuckDB database file (creates the file if it doesn't already exist)
# The database file is named 'air_quality.db' and is stored in the parent directory
con = ddb.connect("../air_quality.db")

In [ ]:
# Create a schema named 'raw' if it doesn't already exist
con.execute("CREATE schema IF NOT EXISTS raw")

In [ ]:
# Create a table named 'air_quality_data' in the 'raw' schema if it doesn't already exist
# This table stores air quality data with various fields such as location, timestamp, and measurements etc
con.execute("""
    CREATE TABLE IF NOT EXISTS raw.air_quality_data (
        location_id BIGINT,
        sensors_id BIGINT,
        "location" VARCHAR,
        "datetime" TIMESTAMP,
        lat DOUBLE,
        lon DOUBLE,
        "parameter" VARCHAR,
        units VARCHAR,
        "value" DOUBLE,
        "month" VARCHAR,
        "year" BIGINT,
        ingestion_datetime TIMESTAMP
    );

""")

In [ ]:
# Insert data from a CSV file stored in an S3 bucket into the 'air_quality_data' table
# The data is filtered by location ID, year, and month, and includes a timestamp for data ingestion
con.execute("""
INSERT INTO raw.air_quality_data
SELECT 
    location_id, 
    sensors_id, 
    "location", 
    "datetime", 
    lat, 
    lon, 
    "parameter", 
    units, 
    "value",
    "month", 
    "year",
    current_timestamp AS ingestion_datetime
FROM read_csv('s3://openaq-data-archive/records/csv.gz/locationid=225393/year=2024/month=01/*.csv.gz');
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
# Close the connection to the DuckDB database
con.close()